### Preamble

In [ ]:
from qiskit import Aer
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np
from qiskit import *
%matplotlib inline
import scipy.linalg as la
import math
import warnings
from qiskit.circuit.library.standard_gates import (HGate, RYGate, RZGate, XGate, ZGate, U1Gate, IGate)
from qiskit.tools.visualization import plot_histogram

import qiskit.circuit.library.standard_gates
from qiskit.converters import circuit_to_instruction
from qiskit.extensions import UnitaryGate

In [ ]:
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%10.4f" % x))

In [ ]:
%%html
<style>
#notebook-container {
    width: 100%;
    background-color: #EEE
}

.code_cell {
   flex-direction: column !important;
}

.code_cell .output_wrapper {
    width: 100%;
    background-color: #FFF
}

.code_cell .input {
    width: 100%;
    background-color: #FFF
}
</style>
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

# WHAT WE WANT

In [ ]:
# # ------------------------------------------------- 2021-01-19, (Previous) PERTURBED COIN UNITARY 
# U1_phase_angle  =   1.5707963267948966 
# U1_RZGate_phi   =   3.1415926535897931 
# U1_RYGate_theta =   2.1208811557885126 
# U1_RZGate_lamb  =   0.0000000000000000 
# U2_phase_angle  =  -0.0000000000000000 
# U2_RZGate_phi   =   3.1415926535897931 
# U2_RYGate_theta =   1.9863951875382797 
# U2_RZGate_lamb  =   3.1415926535897931 
# MID_angle1      =   1.0877844350147556 
# MID_angle2      =   0.1524284793158924 
# V1_phase_angle  =   1.5707963267948966 
# V1_RZGate_phi   =   3.1415926535897931 
# V1_RYGate_theta =   2.6271333499550176 
# V1_RZGate_lamb  =   0.0000000000000000 
# V2_phase_angle  =   1.5707963267948966 
# V2_RZGate_phi   =   3.1415926535897931 
# V2_RYGate_theta =   0.5943894309337333 
# V2_RZGate_lamb  =   0.0000000000000000 

# -------------------------------------------------Input angles: # Ideal Unitary Operator for Perturbed Coin at p=0.2
U1_phase_angle  =  -0.0000000000000000 
U1_RZGate_phi   =   3.1415926535897931 
U1_RYGate_theta =   2.3299691862646794 
U1_RZGate_lamb  =   3.1415926535897931 
U2_phase_angle  =  -0.0000000000000000 
U2_RZGate_phi   =   6.2831853071795862 
U2_RYGate_theta =   0.7703471811449281 
U2_RZGate_lamb  =   0.0000000000000000 
MID_angle1      =   1.3335746805402686 
MID_angle2      =   0.2526979190691310 
V1_phase_angle  =   1.5707963267948966 
V1_RZGate_phi   =   0.0000000000000000 
V1_RYGate_theta =   2.9337768913138365 
V1_RZGate_lamb  =  -3.1415926535897931 
V2_phase_angle  =   1.5707963267948966 
V2_RZGate_phi   =   0.0000000000000000 
V2_RYGate_theta =   1.0788670360654935 
V2_RZGate_lamb  =  -3.1415926535897931 

ThisIsTheUnitary =         [[0.894427190999916,                         0,        -0.447213595499958,     -1.32937475530179e-16],[0.357770876399966,        -0.254397040948809,         0.715541752799933,            0.543398698523],         [                0,         0.905664497538333,     -2.22044604925031e-16,         0.423995068248015],         [0.268328157299975,         0.339196054598412,         0.536656314599949,        -0.724531598030666]]
print(np.around(np.matrix(ThisIsTheUnitary),12))

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr) # Cannot have classical register for wrapping into custom_gate


alpha = MID_angle1
beta = MID_angle2

multiplex = UnitaryGate(np.array([[np.cos(MID_angle1), 0,                 -np.sin(MID_angle1), 0                  ],
                                  [0,                  np.cos(MID_angle2), 0,                  -np.sin(MID_angle2)],
                                  [np.sin(MID_angle1), 0,                  np.cos(MID_angle1), 0                  ],
                                  [0,                  np.sin(MID_angle2), 0,                   np.cos(MID_angle2)]]))
 
##############3# circ.ry(1.287002217587,0)# Rotate to get |sigma_2>
circ.crz(V2_RZGate_lamb,1,0)
circ.cry(V2_RYGate_theta,1,0)
circ.crz(V2_RZGate_phi,1,0)
circ.cu1(V2_phase_angle,1,0)
circ.cx(1,0)
circ.cu1(V2_phase_angle,1,0)
circ.cx(1,0)
circ.x(1)
circ.crz(V1_RZGate_lamb,1,0)
circ.cry(V1_RYGate_theta,1,0)
circ.crz(V1_RZGate_phi,1,0)
circ.cu1(V1_phase_angle,1,0)
circ.cx(1,0)
circ.cu1(V1_phase_angle,1,0)
circ.cx(1,0)
circ.x(1)
circ.unitary(multiplex, [0,1], label='GSVD MTPLX')
circ.crz(U2_RZGate_lamb,1,0)
circ.cry(U2_RYGate_theta,1,0)
circ.crz(U2_RZGate_phi,1,0)
circ.cu1(U2_phase_angle,1,0)
circ.cx(1,0)
circ.cu1(U2_phase_angle,1,0)
circ.cx(1,0)
circ.x(1)
circ.crz(U1_RZGate_lamb,1,0)
circ.cry(U1_RYGate_theta,1,0)
circ.crz(U1_RZGate_phi,1,0)
circ.cu1(U1_phase_angle,1,0)
circ.cx(1,0)
circ.cu1(U1_phase_angle,1,0)
circ.cx(1,0)
circ.x(1)


### Store these gates in custom_gate
custom_gate = circuit_to_instruction(circ)

# circ.measure(0,0)
# circ.measure(1,1)

# circ.draw(output='mpl')
circ.draw()
# circ.draw(output='mpl', interactive=False, filename='Perturbed_coin_20200902_0010.png')


In [ ]:
from qiskit.quantum_info import Operator

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr)
circ.append(custom_gate,[0,1]) # Single custom_gate

mat = Operator(circ)
unitary = mat.data

# print('Real: \n')
print(np.real(np.around(unitary,4)))
print('\n')
# print('Imag: \n')
print(np.imag(np.around(unitary,4)))

In [ ]:
timesteps = 1
qr = QuantumRegister(timesteps+1)
cr = ClassicalRegister(timesteps+1)
circ = QuantumCircuit(qr,cr)
for i in range(0,timesteps):
    circ.append(custom_gate,[timesteps-i-1,timesteps])

circ.barrier()
for i in range(0,timesteps+1):
    circ.measure(i,i)


# circ.measure(0,0)
# circ.measure(1,1)

# circ.measure(timesteps,timesteps)

circ.draw(output='mpl')

## IDEAL -- save data for histogram

In [ ]:
job = execute([circ],
              Aer.get_backend('qasm_simulator'), shots = 1000000)
results = job.result()
counts = results.get_counts()
fig = plot_histogram(counts, figsize=[6,3],title = 'Without noise')

# print('Done')

# ibmq_manila

In [ ]:
from qiskit import QuantumCircuit, execute
from qiskit import IBMQ, Aer
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ

IBMQ.save_account('d1c6ff343b14365b86172d0503f484880269543b4d365f4107e80009f3768f4bb8d5275afe8da21502d921b53b6091222f7a4390eac1fe846a79900110c64e0a', overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q-research')

provider.backends()

### Extract from backend
# Build noise model from backend properties
bknd = 'ibmq_manila'

backend = provider.get_backend(bknd)
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

### Extract from backend -- specific date
# Specifying a date: YYYY, MM, DD
from datetime import datetime, date
# mo = datetime(2020, 12, 12) # <-- Date of generating 61696 basis states files.
mo = datetime(2022, 9, 25)

# How to access properties of a specific date
real_backend = provider.backends.ibmq_manila
prop = real_backend.properties(datetime=mo)
print(prop.last_update_date)

### View noise model
# How to access properties of a specific date
real_backend = provider.backends.ibmq_manila
prop = real_backend.properties(datetime=mo)
print(prop.last_update_date)

# To see the details of the noise model
# noise_model.to_dict()

# Noisy

In [ ]:
job = execute([circ],
              Aer.get_backend('qasm_simulator'),
              noise_model = noise_model,
              basis_gates = basis_gates,
              shots = 1000000)
results = job.result()
counts = results.get_counts()
fig = plot_histogram(counts, figsize=[6,3],title = 'Without noise')

# print('Done')

### 13 BASIS STATES (SINGLE QUBIT)

In [ ]:
BasisStates_169 = [[[1,0],[0,1]],
               
               [[0,1],[1,0]],
               [[0,-1j],[1j,0]],
               [[1,0],[0,-1]],

               [[1/np.sqrt(2)*1,1/np.sqrt(2)*1j],[1/np.sqrt(2)*1j,1/np.sqrt(2)*1]],
               [[1/np.sqrt(2)*1,1/np.sqrt(2)*1],[1/np.sqrt(2)*(-1),1/np.sqrt(2)*1]],
               [[1/np.sqrt(2)*(1+1j),0],[0,1/np.sqrt(2)*(1-1j)]],
               
               [[1/np.sqrt(2)*1,1/np.sqrt(2)*(-1j)],[1/np.sqrt(2)*1j,1/np.sqrt(2)*(-1)]],
               [[1/np.sqrt(2)*1,1/np.sqrt(2)*1],[1/np.sqrt(2)*1,1/np.sqrt(2)*(-1)]],
               [[0,1/np.sqrt(2)*(1-1j)],[1/np.sqrt(2)*(1+1j),0]],
              
               [[1,0],[0,0]],
               [[0.5,0.5],[0.5,0.5]],
               [[0.5,-0.5j],[0.5j,0.5]]]

# BasisStates_169[0]

gatenames = 'BasisState00', 'BasisState01', 'BasisState02', 'BasisState03', 'BasisState04', 'BasisState05', 'BasisState06', 'BasisState07', 'BasisState08', 'BasisState09', 'BasisState10', 'BasisState11', 'BasisState12'


In [ ]:
def GSTBasisGateMeasurements_169(circ):
    #================= START OF 16 MEASUREMENT BASES =================|||
    for j in range(0,16):

        print('[def GSTBasisGateMeasurements]: initial state k=%d, idx_gatenames_1 = %d, idx_gatenames_2 = %d, measurement basis %d' %(k,idx_gatenames_1,idx_gatenames_2,j))

        if j==0: # Identity basis  # Identity basis
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ0 = QuantumCircuit(qr)

            NC0 = circ + circ0
            NC0.measure(0,0)
            NC0.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC0, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) II_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==1: # Identity basis # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ1 = QuantumCircuit(qr)

            NC1 = circ + circ1
            NC1.h(1)
            NC1.measure(0,0)
            NC1.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC1, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) IX_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==2: # Identity basis # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ2 = QuantumCircuit(qr)

            NC2 = circ + circ2
            NC2.sdg(1)
            NC2.h(1)
            NC2.measure(0,0)
            NC2.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC2, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) IY_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==3: # Identity basis # Z basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ3 = QuantumCircuit(qr)

            NC3 = circ + circ3
            NC3.measure(0,0)
            NC3.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC3, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) IZ_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================


        elif j==4:   # X basis# Identity basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ4 = QuantumCircuit(qr)

            NC4 = circ + circ4
            NC4.h(0)
            NC4.measure(0,0)
            NC4.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC4, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) XI_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==5: # X basis # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ5 = QuantumCircuit(qr)

            NC5 = circ + circ5
            NC5.h(0)
            NC5.h(1)
            NC5.measure(0,0)
            NC5.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC5, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) XX_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==6: # X basis # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ6 = QuantumCircuit(qr)

            NC6 = circ + circ6
            NC6.sdg(1)
            NC6.h(1)
            NC6.h(0)
            NC6.measure(0,0)
            NC6.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC6, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) XY_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==7: # X basis  # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ7 = QuantumCircuit(qr)

            NC7 = circ + circ7
            NC7.h(0)
            NC7.measure(0,0)
            NC7.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC7, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) XZ_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================

        elif j==8:  # Y basis # Identity basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ8 = QuantumCircuit(qr)

            NC8 = circ + circ8
            NC8.sdg(0)
            NC8.h(0)
            NC8.measure(0,0)
            NC8.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC8, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) YI_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==9:  # Y basis  # X basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ9 = QuantumCircuit(qr)

            NC9 = circ + circ9
            NC9.h(1)
            NC9.sdg(0)
            NC9.h(0)
            NC9.measure(0,0)
            NC9.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC9, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) YX_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==10: # Y basis  # Y basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ10 = QuantumCircuit(qr)

            NC10 = circ + circ10
            NC10.sdg(0)
            NC10.h(0)
            NC10.sdg(1)
            NC10.h(1)
            NC10.measure(0,0)
            NC10.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC10, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) YY_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==11: # Y basis  # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ11 = QuantumCircuit(qr)

            NC11 = circ + circ11
            NC11.sdg(0)
            NC11.h(0)
            NC11.measure(0,0)
            NC11.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC11, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) YZ_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================

        elif j==12:  # Z basis # Identity basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ12 = QuantumCircuit(qr)

            NC12 = circ + circ12
            NC12.measure(0,0)
            NC12.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC12, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZI_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==13: # Z basis  # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ13 = QuantumCircuit(qr)

            NC13 = circ + circ13
            NC13.h(1)
            NC13.measure(0,0)
            NC13.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC13, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZX_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==14: # Z basis  # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ14 = QuantumCircuit(qr)

            NC14 = circ + circ14
            NC14.sdg(1)
            NC14.h(1)
            NC14.measure(0,0)
            NC14.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC14, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZY_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==15: # Z basis # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ15 = QuantumCircuit(qr)

            NC15 = circ + circ15
            NC15.measure(0,0)
            NC15.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC15, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_BasisState%.2d Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZZ_basis.txt" %(idx_gatenames_1,idx_gatenames_2,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()
    #================= END OF 16 MEASUREMENT BASES ===================||| 

In [ ]:
def GSTBasisGateMeasurements_72(circ,idx_basis_gates_72):
    #================= START OF 16 MEASUREMENT BASES =================|||
    for j in range(0,16):

        print('[def GSTBasisGateMeasurements]: initial state k=%d, idx_gatenames = %d, measurement basis %d' %(k,idx_basis_gates_72,j))

        if j==0: # Identity basis  # Identity basis
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ0 = QuantumCircuit(qr)

            NC0 = circ + circ0
            NC0.measure(0,0)
            NC0.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC0, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) II_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==1: # Identity basis # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ1 = QuantumCircuit(qr)

            NC1 = circ + circ1
            NC1.h(1)
            NC1.measure(0,0)
            NC1.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC1, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) IX_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==2: # Identity basis # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ2 = QuantumCircuit(qr)

            NC2 = circ + circ2
            NC2.sdg(1)
            NC2.h(1)
            NC2.measure(0,0)
            NC2.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC2, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) IY_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==3: # Identity basis # Z basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ3 = QuantumCircuit(qr)

            NC3 = circ + circ3
            NC3.measure(0,0)
            NC3.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC3, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) IZ_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================


        elif j==4:   # X basis# Identity basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ4 = QuantumCircuit(qr)

            NC4 = circ + circ4
            NC4.h(0)
            NC4.measure(0,0)
            NC4.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC4, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) XI_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==5: # X basis # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ5 = QuantumCircuit(qr)

            NC5 = circ + circ5
            NC5.h(0)
            NC5.h(1)
            NC5.measure(0,0)
            NC5.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC5, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) XX_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==6: # X basis # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ6 = QuantumCircuit(qr)

            NC6 = circ + circ6
            NC6.sdg(1)
            NC6.h(1)
            NC6.h(0)
            NC6.measure(0,0)
            NC6.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC6, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) XY_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==7: # X basis  # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ7 = QuantumCircuit(qr)

            NC7 = circ + circ7
            NC7.h(0)
            NC7.measure(0,0)
            NC7.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC7, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) XZ_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================

        elif j==8:  # Y basis # Identity basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ8 = QuantumCircuit(qr)

            NC8 = circ + circ8
            NC8.sdg(0)
            NC8.h(0)
            NC8.measure(0,0)
            NC8.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC8, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) YI_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==9:  # Y basis  # X basis 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ9 = QuantumCircuit(qr)

            NC9 = circ + circ9
            NC9.h(1)
            NC9.sdg(0)
            NC9.h(0)
            NC9.measure(0,0)
            NC9.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC9, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) YX_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==10: # Y basis  # Y basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ10 = QuantumCircuit(qr)

            NC10 = circ + circ10
            NC10.sdg(0)
            NC10.h(0)
            NC10.sdg(1)
            NC10.h(1)
            NC10.measure(0,0)
            NC10.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC10, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) YY_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==11: # Y basis  # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ11 = QuantumCircuit(qr)

            NC11 = circ + circ11
            NC11.sdg(0)
            NC11.h(0)
            NC11.measure(0,0)
            NC11.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC11, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) YZ_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        # =================================================================================================================

        elif j==12:  # Z basis # Identity basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ12 = QuantumCircuit(qr)

            NC12 = circ + circ12
            NC12.measure(0,0)
            NC12.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC12, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZI_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==13: # Z basis  # X basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ13 = QuantumCircuit(qr)

            NC13 = circ + circ13
            NC13.h(1)
            NC13.measure(0,0)
            NC13.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC13, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZX_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==14: # Z basis  # Y basis   
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ14 = QuantumCircuit(qr)

            NC14 = circ + circ14
            NC14.sdg(1)
            NC14.h(1)
            NC14.measure(0,0)
            NC14.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC14, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZY_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()

        elif j==15: # Z basis # Z basis  
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circ15 = QuantumCircuit(qr)

            NC15 = circ + circ15
            NC15.measure(0,0)
            NC15.measure(1,1)

            ### Use qasm aer noise model to run
            job = execute(NC15, 
                          Aer.get_backend('qasm_simulator'), 
                          shots = 8192, 
#                           coupling_map = coupling_map, 
                          basis_gates = basis_gates, 
                          noise_model = noise_model)
            results = job.result()
            counts = results.get_counts()

            for idx_counts in range(0,len(counts)):
                fout = "GST_basisstates/Q_1_Q_2_BasisState%.2d (k=%.2d,j=%.2d) ZZ_basis.txt" %(idx_basis_gates_72,k,j)
                fo = open(fout, "w")
                for v1, v2 in counts.items():
                    fo.write(str(v1) + '  ' + str(v2) + '\n')
                fo.close()
    #================= END OF 16 MEASUREMENT BASES ===================||| 

## TOMOGRAPHY OF REMAINING 72 BASIS STATES (2 QUBITS)

### READ REMAINING 72 BASIS STATES 

In [ ]:
BasisStates_241 = []
for ii in range(169,241):
    BS_2qubits = np.loadtxt('2qubit_basisops/BG_%.3d.txt' %(ii) , dtype=complex, delimiter=',',usecols=range(4)) ## RANGE IS FOR NUMBER OF COLS i.e. NO OF OPERATORS
    BasisStates_241.append(np.matrix(BS_2qubits))

# START OF MONTE CARLO - n-future timesteps

### MonteCarloMeasurementBasis - top register only

In [ ]:
def MonteCarloMeasurementBasis_top_register(index_of_Q, top_register, bottom_register):
    
    #===========================================================================================================
    if index_of_Q == 0: # top = Identity basis # bottom = Identity basis
        pass
    
    elif index_of_Q == 1: # top = Identity basis # bottom = X basis  
        pass
#         circ.h(bottom_register)
    
    elif index_of_Q == 2: # top = Identity basis # bottom = Y basis
        pass
#         circ.sdg(bottom_register)
#         circ.h(bottom_register)

    elif index_of_Q == 3: # top = Identity basis # bottom = Z basis
        pass
    
    #===========================================================================================================
    elif index_of_Q == 4: # top = X basis # bottom = Identity basis 
        circ.h(top_register)

    elif index_of_Q == 5: # top = X basis # bottom = X basis 
        circ.h(top_register)
#         circ.h(bottom_register)

    elif index_of_Q == 6: # top = X basis # bottom = Y basis 
        circ.h(top_register)
#         circ.sdg(bottom_register)
#         circ.h(bottom_register)

    elif index_of_Q == 7: # top = X basis # bottom = Z basis     
        circ.h(top_register)
    
    #===========================================================================================================
    elif index_of_Q == 8: # top = Y basis # bottom = Identity basis     
        circ.sdg(top_register)
        circ.h(top_register)
    
    elif index_of_Q == 9: # top = Y basis # bottom = X basis     
        circ.sdg(top_register)
        circ.h(top_register)
#         circ.h(bottom_register)

    elif index_of_Q == 10: # top = Y basis # bottom = Y basis 
        circ.sdg(top_register)
        circ.h(top_register)
#         circ.sdg(bottom_register)
#         circ.h(bottom_register)

    elif index_of_Q == 11: # top = Y basis # bottom = Z basis    
        circ.sdg(top_register)
        circ.h(top_register)

    #===========================================================================================================
    elif index_of_Q == 12: # top = Z basis # bottom = Identity basis     
        pass
    
    elif index_of_Q == 13: # top = Z basis # bottom = X basis     
        pass
#         circ.h(bottom_register)

    elif index_of_Q == 14: # top = Z basis # bottom = Y basis     
        pass
#         circ.sdg(bottom_register)
#         circ.h(bottom_register)
        
    elif index_of_Q == 15: # top = Z basis # bottom = Z basis     
        pass
    
    #===========================================================================================================   

In [ ]:
def MonteCarloMeasurementBasis_top_and_bottom_registers(index_of_Q, top_register, bottom_register):
    
    #===========================================================================================================
    if index_of_Q == 0: # top = Identity basis # bottom = Identity basis
        pass
    
    elif index_of_Q == 1: # top = Identity basis # bottom = X basis  
        # pass
        circ.h(bottom_register)
    
    elif index_of_Q == 2: # top = Identity basis # bottom = Y basis
        # pass
        circ.sdg(bottom_register)
        circ.h(bottom_register)

    elif index_of_Q == 3: # top = Identity basis # bottom = Z basis
        pass
    
    #===========================================================================================================
    elif index_of_Q == 4: # top = X basis # bottom = Identity basis 
        circ.h(top_register)

    elif index_of_Q == 5: # top = X basis # bottom = X basis 
        circ.h(top_register)
        circ.h(bottom_register)

    elif index_of_Q == 6: # top = X basis # bottom = Y basis 
        circ.h(top_register)
        circ.sdg(bottom_register)
        circ.h(bottom_register)

    elif index_of_Q == 7: # top = X basis # bottom = Z basis     
        circ.h(top_register)
    
    #===========================================================================================================
    elif index_of_Q == 8: # top = Y basis # bottom = Identity basis     
        circ.sdg(top_register)
        circ.h(top_register)
    
    elif index_of_Q == 9: # top = Y basis # bottom = X basis     
        circ.sdg(top_register)
        circ.h(top_register)
        circ.h(bottom_register)

    elif index_of_Q == 10: # top = Y basis # bottom = Y basis 
        circ.sdg(top_register)
        circ.h(top_register)
        circ.sdg(bottom_register)
        circ.h(bottom_register)

    elif index_of_Q == 11: # top = Y basis # bottom = Z basis    
        circ.sdg(top_register)
        circ.h(top_register)

    #===========================================================================================================
    elif index_of_Q == 12: # top = Z basis # bottom = Identity basis     
        pass
    
    elif index_of_Q == 13: # top = Z basis # bottom = X basis     
        # pass
        circ.h(bottom_register)

    elif index_of_Q == 14: # top = Z basis # bottom = Y basis     
        # pass
        circ.sdg(bottom_register)
        circ.h(bottom_register)
        
    elif index_of_Q == 15: # top = Z basis # bottom = Z basis     
        pass
    
    #===========================================================================================================   

### CDF for single unitary operator

In [ ]:
import numpy as np

CDF_O = np.loadtxt('CDF_basisgates_2QUBITS_1GATES_20230223_232918.txt', delimiter=',',usecols=range(4)) ## RANGE IS FOR NUMBER OF COLS i.e. NO OF OPERATORS

CDF_O_index = CDF_O[:,0]
CDF_O_alpha = CDF_O[:,1]
CDF_O_beta  = CDF_O[:,2]
CDF_O_l1  = CDF_O[:,3]

print('CDF_O_index = ', CDF_O_index[0:13], '...')
print('CDF_O_alpha = ', CDF_O_alpha[0:13], '...')
print('CDF_O_beta  = ', CDF_O_beta[0:13], '...')
print('CDF_O_l1    = ', CDF_O_l1[0:13], '...')
# print('CDF_O_l2 = ', CDF_O_l2)

CDF_Q = np.loadtxt('CDF_Q_2QUBITS_1GATES_20230223_232919.txt', delimiter=',',usecols=range(1))
print('\nCDF_Q = ', CDF_Q[0:13], '...')

CDF_rho = np.loadtxt('CDF_rho_2QUBITS_1GATES_20230223_232919.txt', delimiter=',',usecols=range(11))
print('\nCDF_rho = (columnwise) \n',CDF_rho)


### Monte Carlo for single unitary operator - tiled n-future timesteps

In [ ]:
timesteps = 1

In [ ]:
from datetime import datetime
import random

for RUN in range(0,200): # Supposed to run for 1000 as per the paper #400 for 100,000 MC trials

    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%Y%m%d_%H%M%S")

    MC_results = np.linspace(1,3+3*timesteps, 3+3*timesteps)
    # MC_results = np.array(['timestep','idx_run','index_rho_vec','index_of_CDF_O_l1_vec','index_of_Q_vec','Measurement_outcome'])

    for idx_run in range(1,1001):#501
        
        qr = QuantumRegister(timesteps+1)
        cr = ClassicalRegister(timesteps+1)
        circ = QuantumCircuit(qr,cr)
        
        print('===========================================\nRUN %d, MC trial %d' %(RUN, idx_run))
        
        index_of_rho_vec = np.zeros((1,timesteps))
        index_of_Q_vec = np.zeros((1,timesteps))
        index_of_CDF_O_l1_vec = np.zeros((1,timesteps))
        
        for idx_timesteps in range(0,timesteps):
            
            top_register = timesteps - idx_timesteps-1
            bottom_register = timesteps
            
            #################################################
            ### SAMPLE FROM CDFS, GENERATE RANDOM NUMBERS ###
            #################################################
            
#             index_of_rho = np.where(CDF_rho[:,1] >= np.random.rand())
            index_of_rho = np.where(CDF_rho[:,idx_timesteps] >= np.random.rand())
            index_of_rho = index_of_rho[0][0]

            index_of_Q = np.where(CDF_Q >= np.random.rand())
            index_of_Q = index_of_Q[0][0]

            RAND_Ol1 = np.random.rand()
            index_of_CDF_O_l1 = np.where(CDF_O_l1 >= RAND_Ol1)
            index_of_CDF_O_l1 = index_of_CDF_O_l1[0][0]

            print('Future timestep t = %d | Index of rho = %d | Index of O_l1 = %.2d | Index of Q = %d ' %(idx_timesteps+1, index_of_rho, index_of_CDF_O_l1, index_of_Q))
            index_of_rho_vec[0,idx_timesteps] = index_of_rho
            index_of_Q_vec[0,idx_timesteps] = index_of_Q
            index_of_CDF_O_l1_vec[0,idx_timesteps] = index_of_CDF_O_l1
            
#             print(index_of_CDF_O_l1_vec)

            ###########################################################
            ### RUN CIRCUIT WITH ADDITIONAL BASIS GATES FOR Unitary ### 
            ###########################################################

            if index_of_CDF_O_l1 < 169:

                #===============#=========================#
                # 16 SCENARIOS  #  13^2 BASIS OPERATIONS  #  -- BIG ENDIAN NOTATION FOR THE 13 BASIS GATES TENSORED PRODUCTS
                #==================================================================================

                if  int(CDF_O_beta[index_of_CDF_O_l1]) < 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) < 10: ### SCENARIO 1
                    circ.append(custom_gate,[top_register, bottom_register]) # O_l1
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_beta[index_of_CDF_O_l1])],[top_register],label='%s' %(gatenames[int(CDF_O_beta[index_of_CDF_O_l1])])) # q_2 on top row
                    circ.unitary(BasisStates_169[int(CDF_O_alpha[index_of_CDF_O_l1])],[bottom_register],label='%s' %(gatenames[int(CDF_O_alpha[index_of_CDF_O_l1])])) # q_1 on bottom row
                    circ.barrier()

                #==================================================================================

                ### PARTIALLY REPREPARE QUBIT_1

                elif int(CDF_O_beta[index_of_CDF_O_l1])  < 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 10: ### SCENARIO 2
                    # Reprepare qubit_1 (bottom row) in state |+>, 
                    # Leave qubit_2 (top row) to be operated on by BasisStates_169[CDF_O_beta[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_beta[index_of_CDF_O_l1])] ,[top_register],label='%s' %(gatenames[int(CDF_O_beta[index_of_CDF_O_l1])])) # q_2 on top row
                    circ.reset(bottom_register)
                    circ.h(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) < 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 11: ### SCENARIO 3
                    # Reprepare qubit_1 (bottom row) in state |y+>, 
                    # Leave qubit_2 (top row) to be operated on by BasisStates_169[CDF_O_beta[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_beta[index_of_CDF_O_l1])] ,[top_register],label='%s' %(gatenames[int(CDF_O_beta[index_of_CDF_O_l1])])) # q_2 on top row
                    circ.reset(bottom_register)
                    circ.h(bottom_register)
                    circ.s(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) < 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 12: ### SCENARIO 4
                    # Reprepare qubit_1 (bottom row) in state |0>, 
                    # Leave qubit_2 (top row) to be operated on by BasisStates_169[CDF_O_beta[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_beta[index_of_CDF_O_l1])] ,[top_register],label='%s' %(gatenames[int(CDF_O_beta[index_of_CDF_O_l1])])) # q_2 on top row
                    circ.reset(bottom_register)
                    circ.barrier()

                #==================================================================================

                ### PARTIALLY REPREPARE QUBIT_2

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) < 10: ### SCENARIO 5
                    # Reprepare qubit_2 (top row) in state |+>, 
                    # Leave qubit_1 (bottom row) to be operated on by BasisStates_169[CDF_O_alpha[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_alpha[index_of_CDF_O_l1])] ,[bottom_register],label='%s' %(gatenames[int(CDF_O_alpha[index_of_CDF_O_l1])])) # q_1 on bottom row
                    circ.reset(top_register)
                    circ.h(top_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 11 and int(CDF_O_alpha[index_of_CDF_O_l1]) < 10: ### SCENARIO 6
                    # Reprepare qubit_2 (top row) in state |y+>, 
                    # Leave qubit_1 (bottom row) to be operated on by BasisStates_169[CDF_O_alpha[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_alpha[index_of_CDF_O_l1])] ,[bottom_register],label='%s' %(gatenames[int(CDF_O_alpha[index_of_CDF_O_l1])])) # q_1 on bottom row
                    circ.reset(top_register)
                    circ.h(top_register)
                    circ.s(top_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 12 and int(CDF_O_alpha[index_of_CDF_O_l1]) < 10: ### SCENARIO 7
                    # Reprepare qubit_2 (top row) in state |y+>, 
                    # Leave qubit_1 (bottom row) to be operated on by BasisStates_169[CDF_O_alpha[RAND_Ol1]].
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    circ.unitary(BasisStates_169[int(CDF_O_alpha[index_of_CDF_O_l1])] ,[bottom_register],label='%s' %(gatenames[int(CDF_O_alpha[index_of_CDF_O_l1])])) # q_1 on bottom row
                    circ.reset(top_register)
                    circ.barrier()

                #==================================================================================

                ### FULLY REPREPARE QUBIT_1 AND QUBIT_2

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 10: ### SCENARIO 8
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |+>, qubit_1 (bottom row) in |+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.h(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 11: ### SCENARIO 9
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |+>, qubit_1 (bottom row) in |y+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.h(bottom_register)
                    circ.s(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 10 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 12: ### SCENARIO 10
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |+>, qubit_1 (bottom row) in |0>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.barrier()

                #==================================================================================

                ### FULLY REPREPARE QUBIT_1 AND QUBIT_2

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 11 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 10: ### SCENARIO 11
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |y+>, qubit_1 (bottom row in |+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.s(top_register)
                    circ.h(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 11 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 11: ### SCENARIO 12
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |y+>, qubit_1 (bottom row in |y+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.s(top_register)
                    circ.h(bottom_register)
                    circ.s(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 11 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 12: ### SCENARIO 13
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |y+>, qubit_1 (bottom row in |0>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(top_register)
                    circ.s(top_register)
                    circ.barrier()

                #==================================================================================

                ### FULLY REPREPARE QUBIT_1 AND QUBIT_2

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 12 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 10: ### SCENARIO 14
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |0>, qubit_1 (bottom row) in |+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 12 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 11: ### SCENARIO 15
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |0>, qubit_1 (bottom row) in |y+>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.h(bottom_register)
                    circ.s(bottom_register)
                    circ.barrier()

                elif int(CDF_O_beta[index_of_CDF_O_l1]) == 12 and int(CDF_O_alpha[index_of_CDF_O_l1]) == 12: ### SCENARIO 16
                    circ.append(custom_gate,[top_register, bottom_register])
                    circ.barrier()
                    # Reprepare qubit_2 (top row) in |0>, qubit_1 (bottom row) in |0>
                    circ.reset(top_register)
                    circ.reset(bottom_register)
                    circ.barrier()

            elif index_of_CDF_O_l1 >= 169:

                circ.append(custom_gate,[top_register, bottom_register])
                circ.barrier()
                circ.unitary(BasisStates_241[index_of_CDF_O_l1-169],[top_register, bottom_register],label='BasisStates %d' %(index_of_CDF_O_l1))
                circ.barrier()

                                             
            ######################### 
            ### MEASUREMENT BASIS ###
            #########################
            
            # Call function to choose basis
            if idx_timesteps < timesteps-1:
                MonteCarloMeasurementBasis_top_register(index_of_Q, top_register, bottom_register)
                circ.barrier()
            elif idx_timesteps == timesteps-1:
                MonteCarloMeasurementBasis_top_and_bottom_registers(index_of_Q, top_register, bottom_register)
                circ.barrier()
            
#         circ.barrier()
        for idx_timesteps in range(0,timesteps+1):
            circ.measure(idx_timesteps,idx_timesteps)    
            

        #######################################
        ### Use qasm aer noise model to run ###
        #######################################

        backend_qasm = Aer.get_backend('qasm_simulator')
        job_qasm = execute(circ,
                           backend_qasm,
#                            coupling_map = coupling_map,
                           basis_gates = basis_gates,
                           noise_model = noise_model,
                           shots=1)

        result_qasm = job_qasm.result()
        counts = result_qasm.get_counts()
        print('Measurement outcome: ',int(list(counts)[0]))

        MC_run = np.hstack(( [[timesteps]], [[idx_run]], index_of_rho_vec, index_of_CDF_O_l1_vec, index_of_Q_vec, [[int(list(counts)[0])]] ))
        MC_results = np.vstack(( MC_results , MC_run ))

    mat = np.matrix(MC_results)
    print(mat)
    with open('L%d/outfile_2qubit_ActualPerturbedCoinUnitary_c0_%d_%s.txt' %(timesteps,RUN,timestampStr) ,'wb') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%6d')

# Audio(sound_file, autoplay=True)

# END OF MONTE CARLO - n-future timesteps